In [5]:
pip install request

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [6]:
pip install beautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
driver.get("https://www.idus.com") #tab[0]
driver.execute_script('window.open("about:blank", "_blank");') #창 1개더 열어놓기 tab[1]
driver.execute_script('window.open("about:blank", "_blank");') 
tabs = driver.window_handles
time.sleep(3)

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

img_List=[]
with open ("ourdus_product.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    time.sleep(3)
    driver.switch_to_window(tabs[0])
    scroll()
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    myWriter.writerow(["카테고리아이디","작품아이디","작품이름","작가이름",'가격','적립금','구매후기수','즐겨찾기수','옵션수'])
       
    category_list =  soup.findAll('div',{'class':'ui_grid__cols--6'}) #모든 카테고리 목록
    
    cid = 0
    pid=0
    for i in category_list:
        cid+=1
        if cid==2:
            break
        category = i.findAll('div',{'class':'ui_grid__item'})
        category_id = cid
        category_info = i.find('a',{'class':'area-txt'})
        category_name = category_info.text
        category_url = "https://www.idus.com/" + str(category_info["href"])
        
        driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
        driver.get(category_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        class_list = soup.findAll('div',{'class':'ui_grid__item'})
        
        for j in class_list:
            pid+=1
            if pid==15: 
                break
            info_url = j.find({'a':'data-v-27ea337a'})
            info_url = "https://www.idus.com/"+ str(info_url["href"])
            img_url = j.find('div',{'class':'product-thumb-img'})
            img_url = img_url["style"]
            start=img_url.find('\"')
            end = img_url.find('\"',start+1)
            img_URL = img_url[start+1:end]
            img_List.append(img_URL)
            
            #img_URL로 로컬 저장소에 사진 저장
            with urlopen(img_URL) as f:
                with open('./Ourdus/product_img/'+ str(pid)+ '.jpg',"wb") as h:
                    img=f.read()
                    h.write(img)
            
            driver.switch_to_window(tabs[2]) #세번째 탭으로 이동
            scroll()
            driver.get(info_url)
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            
            writer_name = soup.find('span',{'class':'artist_card__label'}).get_text(strip=True)
            writer_name = writer_name.replace(" ","")
            
            product_name = soup.find('h2',{'class':'sticky_aside__produc-title'}).text
            
            detail_info = soup.find('div',{'class':'product-detail-info'})
            
            product_price = detail_info.find('strong',{'class':'sold-price'}).text.replace(",","")
            product_price=int(product_price)
            
            try:
                product_hit = detail_info.find('p',{'class':'star-count-text'}).get_text(strip=True)
                product_hit = product_hit.replace(",","")
                product_hit=int(product_hit)
            except:
                product_hit=0

            try:
                product_purchase = detail_info.find('span',{'id':'count_up'}).text
                product_purchase = product_purchase.replace(" ","").replace("명","").replace(",","")
                product_purchase = int(product_purchase)
            except:
                product_purchase=0
            
            try:
                point_info = detail_info.find('td',{'class':'data-row__content'})
                product_point = point_info.find({'span':'data-v-9e98616a'},{'class':'bold-txt'}).text
                product_point = product_point.replace(" ","").replace("최","").replace("대","").replace("P","").replace(",","")
                product_point=int(product_point)
            except:
                product_point=0
            
            try:
                product_rate = detail_info.find('span',{'class':'ui_rating fr'})
                product_rate = product_rate["data-value"]
            except:
                product_rate=0
            
            try:
                product_review = detail_info.find('div',{'style':'margin-left: 3px;'}).text
                product_review = product_review.replace(")","").replace("(","")
                product_review = int(product_review)
            except:
                product_review=0
            
#             delivery_info = detail_info.find('td',{'class':'data-row__content'})
#             product_delivery = delivery_info.find({'span':'data-v-9e98616a'}).get_text(strip=True)
#             product_delivery = product_delivery.replace(",","").replace("원","")
            
            product_description = soup.findAll('p',{'class':'para'})
            
            save = [cid,pid,product_name,writer_name,product_price,product_point,product_review,product_hit,3]
            print(save)
            myWriter.writerow(save)
            
with open ("ourdus_product_image.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite) 
    for img in img_List:
        myWriter.writerow(img)

driver.close()
    
 

<ipython-input-3-9655819ad17b>:37: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])
<ipython-input-3-9655819ad17b>:58: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
<ipython-input-3-9655819ad17b>:84: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[2]) #세번째 탭으로 이동


[1, 1, '❤설날선물세트할인 (2/8발송)❤ 딸기청 딸기우유 라떼', '초코써니수제청수제과일청', 12900, 451, 1443, 18993, 3]
[1, 2, ' 설 전 수령 ❤ 포장무료 ❤ 바른 딸기청 ', '포시즌테이블', 12900, 451, 1111, 6230, 3]
[1, 3, '[설날선물]🌸5종+1종랜덤증정🌸미니꽃차5종세트🌸', '플로썸', 21250, 743, 570, 10943, 3]
[1, 4, '⚡설날전수령⚡발렌타인설날선물더치커피6종선물세트 3+1 ', '그:때커피', 10300, 360, 584, 6152, 3]
[1, 5, '[⭐주말출고⭐]알록달록💕12종더치커피세트', '보르딘', 17900, 626, 1711, 16639, 3]
[1, 6, '주말출고설날선물세트발렌타인(1+1+1)더치커피', '그:때커피', 12500, 437, 2172, 9530, 3]
[1, 7, '시음공짜❤️고소한 에스프레소 커피 원액 500ml', '소블링커피', 11980, 419, 981, 4139, 3]
[1, 8, '[⭐주말출고⭐]알록달록💕24종더치커피세트', '보르딘', 35500, 1242, 1366, 17479, 3]
[1, 9, '[설날선물]🌸꽃차집게증정+꽃차5종샘플세트🌸', '보나플로', 25000, 875, 53, 574, 3]
[1, 10, '설날전수령⚡발렌타인.설날선물세트 더치커피 문구제작파우치', '그:때커피', 1000, 35, 97, 2985, 3]
[1, 11, '[💖설날명절선물세트] 주말출고⚡️ 더치커피선물 🎁', '마켓컴퍼니', 37000, 1295, 78, 2715, 3]
[1, 12, '설날선물 더치커피 콜드브루원액세트', '마이더치커피', 9900, 346, 1621, 9612, 3]
[1, 13, '🔥오늘마감❗설날특가🔥대용량 더치커피 발렌타인 선물', '엘리펀트커피랩', 10500, 367, 2783, 7632, 3]
[1, 14, '설날선물세트❤수제청❤설날 선물세트❤답례품❤설날❤', '땡기는공방수제청', 10000, 